# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [15]:
import numpy as np
import os, io, base64
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util
from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'inference_graphs'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'object_detection.pbtxt')

NUM_CLASSES = 1

## Download Model

In [5]:
"""
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
"""

"\nopener = urllib.request.URLopener()\nopener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)\ntar_file = tarfile.open(MODEL_FILE)\nfor file in tar_file.getmembers():\n  file_name = os.path.basename(file.name)\n  if 'frozen_inference_graph.pb' in file_name:\n    tar_file.extract(file, os.getcwd())\n"

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [8]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [38]:
img64 = '...'

In [29]:
min_score_thresh = 0.7

# Detection

In [39]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        msg = base64.b64decode(img64)
        buf = io.BytesIO(msg)
        img = Image.open(buf)
        image_np = load_image_into_numpy_array(img)
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})

        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)
        img = Image.fromarray(image_np)
        print [category_index.get(value) for index,value in enumerate(classes[0]) if scores[0,index] > min_score_thresh]
        #dclasses = list(np.squeeze(classes[0]).astype(np.int32))

[{'id': 1, 'name': u'ps4'}]


In [22]:
width, height = img.size

In [26]:
boxes[0][0][0]*height

203.09428811073303

In [28]:
for i, coord in enumerate(boxes[0]):
    ymin = boxes[0][i][0]*height
    xmin = boxes[0][i][1]*width
    ymax = boxes[0][i][2]*height
    xmax = boxes[0][i][3]*width
    print 'Top left'
    print (xmin,ymin,)
    print 'Bottom right'
    print (xmax,ymax)
    print "\n\n"

Top left
(6.9363307952880859, 203.09428811073303)
Bottom right
(192.55954742431641, 420.3985161781311)



Top left
(563.94123077392578, 374.9963675737381)
Bottom right
(635.85651397705078, 409.69832718372345)



Top left
(227.22040176391602, 408.04307556152344)
Bottom right
(295.38629531860352, 426.0)



Top left
(482.15915679931641, 124.51885449886322)
Bottom right
(626.65218353271484, 387.03197586536407)



Top left
(85.753726959228516, 0.0)
Bottom right
(380.92910766601562, 174.18881964683533)



Top left
(177.24678039550781, 0.0)
Bottom right
(280.08052825927734, 149.99841499328613)



Top left
(563.58448028564453, 402.69875764846802)
Bottom right
(637.05066680908203, 426.0)



Top left
(259.16603088378906, 407.32710921764374)
Bottom right
(331.98825836181641, 426.0)



Top left
(235.7708740234375, 268.45756924152374)
Bottom right
(532.50473022460938, 407.76882112026215)



Top left
(74.445686340332031, 18.926571696996689)
Bottom right
(416.55662536621094, 119.32439768314362)



To

In [ ]:
bboxes = boxes[0]
print bboxes

In [ ]:
print category_index

In [ ]:
from scipy import stats

In [ ]:
new_classes = np.squeeze(classes[0]).astype(np.int32)

In [ ]:
classes[0]

In [ ]:
c = list(new_classes)

In [ ]:
max(list(new_classes), key=list(new_classes).count)

In [ ]:
for bb in bboxes:
    print bb[0]*im_height

In [ ]:
bbox[0]*im_width

In [ ]:
bbox[1]*im_height

In [ ]:
np.squeeze(boxes)